# _EXPERT CHECK-IN_ options

The [getting started notebook](./getting_started.ipynb) shows the most simple approach to check in time series for _EXPERT_. In this notebook you can find various ways to check in your data.

## Prerequisites for running this notebook
Login with your account and create a forecasting configuration. For details and configuration options please have a look at the [getting started notebook](./getting_started.ipynb) or other more advanced notebooks.

In [ ]:
from futureexpert import (DataDefinition,
                          ExpertClient,
                          FileSpecification,
                          ForecastingConfig,
                          MethodSelectionConfig,
                          PreprocessingConfig,
                          ReportConfig,
                          TsCreationConfig)

client = ExpertClient.from_dotenv()
fc_report_config = ReportConfig(title='My first report',
                                forecasting=ForecastingConfig(fc_horizon=7),
                                preprocessing=PreprocessingConfig(),
                                method_selection=MethodSelectionConfig(number_iterations=8))

## Option 1: Check in your time series via the _future_ frontend

Check in your time series for _EXPERT_ in the _future_ frontend https://now.future-forecasting.de:

1. Upload your data using Python or the future frontend.
2. Navigate to 'My Data' in future frontend.
3. Start the _CHECK-IN_ process from a file.
4. Follow along the _CHECK-IN_ steps in the frontend.
5. Click on 'Continue with...' and 'Forecast via _EXPERT_' and copy the version ID.

You can then use your checked in data for forecasting in _EXPERT_.

In [ ]:
ts_version = 'TBD' # replace with your version

# Request forecast creation and retrieve the id for status requests and result retrieval
forecast_identifier_from_version_id = client.start_forecast(version=ts_version, config=fc_report_config)

### Option 2: Use the _CHECK-IN_ configuration file from the _future_ frontend

#### Get a _CHECK-IN_ configuration file

Use a _CHECK-IN_ configuration file generated during the _CHECK-IN_ process in the _future_ frontend https://now.future-forecasting.de:

1. Upload your data using Python or the future frontend.
2. Navigate to 'My Data' in future frontend.
3. Start the _CHECK-IN_ process from a file.
4. Follow along the _CHECK-IN_ steps in the frontend.
5. Download the configuration at the end.
6. Adjust the file pathe in the below code block.

You can then reuse your uploaded data with the downloaded _CHECK-IN_ configuration for forecasting in _EXPERT_ or even reuse the _CHECK-IN_ configuration for new data.

#### Invoke forecast computation

In [ ]:
# Request forecast creation and retrieve the id for status requests and result retrieval
forecast_identifier_from_config_file = client.start_forecast_from_raw_data(raw_data_source='../example_data/bicycle_data.csv',
                                                                           config_fc=fc_report_config,
                                                                           config_checkin='../example_data/config_bicycle_data.json')

### Option 3: Configure _CHECK-IN_ in code

Configure the data definition and time series creation in code.

**Note:** The prefered way to configure the data defintion is using the _future_ frontend at https://now.future-forecasting.de, as it provides the full range of available features, e.g. rules for handling detected data issues, and inclusion and exclusion rules for data preparation.

In [3]:
import futureexpert.checkin as checkin

data_definition = DataDefinition(
    date_columns=checkin.DateColumn(name='Date', format='%Y-%m-%d'),
    value_columns=[checkin.ValueColumn(name='value')],
    group_columns=[checkin.GroupColumn(name='Name')])

ts_creation_config = TsCreationConfig(time_granularity='daily',
                                      description='bicycle count',
                                      grouping_level=['Name'],
                                      value_columns_to_save=['value'],
                                      missing_value_handler='setToZero')


# Request forecast creation and retrieve the id for status requests and result retrieval
forecast_identifier_from_code_definition = client.start_forecast_from_raw_data(raw_data_source='../example_data/bicycle_data.csv',
                                                                               data_definition=data_definition,
                                                                               config_ts_creation=ts_creation_config,
                                                                               config_fc=fc_report_config,
                                                                               file_specification=FileSpecification(delimiter=';', decimal='.'))

### Option 3b: Directly upload your data as a dataframe

This is almost identical to option 3 but allows you to start from a dataframe. This dataframe will be uploaded as a csv file and stored in your data.

In [ ]:
import pandas as pd

raw_data_df = pd.read_csv('../example_data/bicycle_data.csv', sep=';', decimal='.')

# Request forecast creation and retrieve the id for status requests and result retrieval
forecast_identifier_from_dataframe = client.start_forecast_from_raw_data(raw_data_source=raw_data_df,
                                                                         data_definition=data_definition,
                                                                         config_ts_creation=ts_creation_config,
                                                                         config_fc=fc_report_config,
                                                                         file_specification=FileSpecification(delimiter=';', decimal=''))

## Get version_id of previous CHECK-IN processes
If you have checked in your time series via the _future_ frontend but forgot to copy the version id or you just want to check which data you have already checked in: with `get_ts_versions` you can check all currently checked-in data and receive the version_id.

In [ ]:
ts_overview = client.get_ts_versions()
ts_overview

## Retrieve time series data after CHECK-IN
With `get_time_series` you can retrieve the data that you have previously checked in.

In [ ]:
results = client.get_time_series(version_id=ts_overview.loc[0, 'version_id'])

## Inspect the forecasting results

In this case, we first wait for all forecasting results before accessing the results.

In [ ]:
import time

# Watch the current status of the forecasting report
while not all(client.get_report_status(id=forecast_identifier).is_finished for forecast_identifier in [forecast_identifier_from_version_id, forecast_identifier_from_config_file, forecast_identifier_from_code_definition, forecast_identifier_from_dataframe]):
    print('Waiting another 30 seconds to finish forecasting...')
    time.sleep(30)  # Wait between status requests

# Retrieve the final results
results_from_version_id = client.get_fc_results(
    id=forecast_identifier_from_version_id, include_backtesting=True, include_k_best_models=2)
results_from_config_file = client.get_fc_results(
    id=forecast_identifier_from_config_file, include_backtesting=True, include_k_best_models=2)
results_from_code_definition = client.get_fc_results(
    id=forecast_identifier_from_code_definition, include_backtesting=True, include_k_best_models=2)
results_from_dataframe = client.get_fc_results(
    id=forecast_identifier_from_dataframe, include_backtesting=True, include_k_best_models=2)